--------------------------------------------------------------
# **Adverse Outcomes Analysis**
--------------------------------------------------------------

---------------------
## **Context**
---------------------



----------------------
## **Objective**  
----------------------



------------------------------------
## **Dataset Description**
------------------------------------

The data contains information on employees' demographic details, work-related metrics, and attrition flag.

* **Visit CSN** - Unique ED Tag




## **Importing the required libraries and overview of the dataset**

In [1]:
import pandas as pd

import numpy as np

import matplotlib.pyplot as plt

import seaborn as sns

# To scale the data using z-score
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split

# Algorithms to use
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.linear_model import LogisticRegression

from sklearn.neighbors import KNeighborsClassifier

# Metrics to evaluate the model
from sklearn import metrics

from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve,recall_score, roc_auc_score

from sklearn import tree

from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import BaggingClassifier

from sklearn.ensemble import RandomForestClassifier

# For tuning the model
from sklearn.model_selection import GridSearchCV

# Importing PCA and t-SNE
from sklearn.decomposition import PCA

from sklearn.manifold import TSNE

# To ignore warnings
import warnings
warnings.filterwarnings("ignore")

### **Loading the dataset**

Merge Datasets

In [ ]:
# Reading the dataset
# LEP 1. Merge Lep 1 + Vitals/ICD10. Aidan 2020, 2021
df1 = pd.read_csv('LEP2020.csv')
df2 = pd.read_csv('LEP2021.csv')

df3 = pd.concat([df1,df2])


# LEP 2. Add on to LEP with previous visits and social history
df4 = pd.read_excel('LEP2SEMN.2020.2021.xlsx')


# Frequent Visitors by Year
#df3 = pd.read_csv('3.csv')




In [ ]:
# merge the data sets

df = df3.merge(df4, on = 'csn', how = 'outer').fillna(np.nan)

In [ ]:
#df = df4.merge(df3, on = 'mrn', how = 'outer').fillna(np.nan)

In [ ]:
# create a copy
dfcopy = df.copy()

In [ ]:
#Reset
df = dfcopy

In [ ]:
# replace spaces in column title
df.columns = df.columns.str.replace(' ', '')
df.columns = df.columns.str.replace('(', '')
df.columns = df.columns.str.replace(')', '')

In [ ]:
df.head()

In [ ]:
#(rows,columns)
df.shape

### **Checking the info of the data**

In [ ]:
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)


df.info()

# Define Outcome of interest

In [ ]:
# Define Variable
df['outcome']=df['return_7day']

In [ ]:
# Drop Outcome Variable
df = df.drop(['return_7day'], axis = 1)

In [ ]:
df['arrival_dt'] = pd.to_datetime(df['arrival_dt'])

In [ ]:
from datetime import datetime, timedelta
df['dtstart365'] = (df['arrival_dt']- timedelta(days =365))

In [ ]:
df = df.reset_index()
df['arrival_dt2'] = (df['arrival_dt'])

In [ ]:

x = 365

df['Date_Start'] = df['arrival_dt'] - pd.offsets.Day(x)
df=df.set_index('arrival_dt').sort_index()
df['Uniquecsn_x_days_ago'] = df.apply(lambda x: df.loc[x.Date_Start : x.arrival_dt2].mrn.nunique(), axis=1)
df = df.reset_index()

In [ ]:
x = np.where(df['arrival_dt'] > df['dtstart365'])

df100 = df

df100['x']= np.where(df['arrival_dt'] > df['dtstart365'])

df100

#df['visit365days'] = np.where(df['arrival_dt'] > df['dtstart365'])

#df['WeekBB'] = np.where(df.DaysBetweenVisitsReturnEDVisit < 8,1,0)


#df.where(df.dtstart365 < df.arrival_dt)

In [ ]:
df1 = df[df.groupby("mrn")['csn'].transform('size') > 1]

In [ ]:
df['visit365days']=df.groupby(by = 'mrn').value_counts()

In [ ]:
df['visit365days']=df.groupby(by = 'mrn').filter(df['arrival_dt'] > df['dtstart365']).value_counts()

In [ ]:
df['mrn'].where(df['arrival_dt'] > df['dtstart365']).

In [ ]:
df.mrn.value_counts()

In [ ]:
df['mrn'].value_counts(sort=False).

In [ ]:
# Create new variables


df = df.groupby(by='domain', as_index=False).agg({'ID': pd.Series.nunique})

df['col1'].value_counts(sort=True)

df['Account_Type].unique() 
# f I want to now for each group of A (x1, x2, x3) how many unique values there are in B which in C are greater than 2, and assign this to a new column D 


df['Visit365days']=df['mrn'].value_counts(sort=False).where(df['EDArrival']<365)
 
df['Death30Day']= np.where(df.ED2DeathDays <31,1,0)
    
    df['DateandTimeofDeath-MC']-df['EDArrival']).dt.days









df['D'] = df['A'].map(df[df['C']>2].groupby('A')['B'].nunique()).fillna(0)


df['D'] = df['A'].map(df.query('C > 2').groupby('A')['B'].nunique()).fillna(0).astype(int)
print (df)


df['num_edenc_lastyr'] = df

.stack().groupby(level=0).nunique()


df['ED2DeathDays']=(df['DateandTimeofDeath-MC']-df['EDArrival']).dt.days

df['Death30Day']= np.where(df.ED2DeathDays <31,1,0)

#df['ED2Death30Days']= np.where(df.ED2DeathDays <31,1,0)


#df['AMA'] = np.where(df['FinalEDDisposition'].str.contains('AMA'),1,0)

#df['WeekBB'] = np.where(df.DaysBetweenVisitsReturnEDVisit < 8,1,0)

#df['ED2Death7Days']= np.where(
    #(df.ED2DeathDay <8) & (df.X == 0),1,0)





In [ ]:
# Describe Feature of Interest
df.visit365days.describe()

In [ ]:
# Making a list of all categorical variables
cat_col = list(df.select_dtypes("object").columns)

# Printing count of each unique value in each categorical column
for column in cat_col:
    print(df[column].value_counts(normalize = False))
    print("-" * 50)

In [ ]:
# Making a list of all categorical variables
cat_col = list(df.select_dtypes("object").columns)

# Printing count of each unique value in each categorical column
for column in cat_col:
    print(df[column].value_counts(normalize = True))
    print("-" * 50)

# Data Cleaning

## **Applying Inclusion and Exclusion Criteria**

### **Inclusion Criteria**

In [ ]:
# Only Keep Rows that Have These Strings

df = df[df.ed_disposition.isin(['Discharge'])]
# drop the column
df = df.drop(['ed_disposition'], axis = 1)


df = df[df.ed_region.isin(['MCHS SE MN'])]
#drop the column
df = df.drop(['ed_region'], axis = 1)




#df = df[df['FinalEDDisposition'] == 1]

#df.loc[df['FinalEDDisposition_Discharge'] == 1]


#df = df.drop(df['FinalEDDisposition_Discharge'] == 1, axis=0)

### **Exclusion Criteria**

In [ ]:
# Drop rows that contain these strings
#df = df[~df.ED.str.contains("return") == True]
df = df[~df.ed.str.contains("MCHS OWOD ED") == True]
df = df.drop(df[df.age < 18].index)

## Dropping Variables

In [ ]:
# Dropping the columns that are roughly unique patient identifiers. 
df = df.drop(['csn'], axis = 1)
df = df.drop(['mrn'], axis = 1)

In [ ]:
# Checking the number of unique values in each column
df.nunique()

In [ ]:
#Dropping Variables that Only Have 1 or 0 Values
#df = df.drop([ 'PreferredCareLanguage', 'Unnamed:38'], axis = 1)

In [ ]:
# Dropping Variables that are in the definition of the outcome
df = df.drop([ 'return_3day'], axis = 1)

In [ ]:
# Dropping Variables that  are in the future with respect to the outcome. 
df = df.drop(['csn_return'], axis = 1)
df = df.drop(['ed_return'], axis = 1)
df = df.drop(['return_ed_match'], axis = 1)
df = df.drop(['hours_to_return'], axis = 1)
df = df.drop(['esi_return'], axis = 1)
df = df.drop(['cc_cat_return'], axis = 1)
df = df.drop(['chief_complaint_return'], axis = 1)
df = df.drop(['primary_dx_return'], axis = 1)
df = df.drop(['disposition_return'], axis = 1)
df = df.drop(['dispo_return_cat'], axis = 1)


In [ ]:
#drop variables with high degree is missingness
#df = df.drop(['YearsofEducation'], axis = 1)


In [ ]:
# Making a list of all datetime variables
datetime_col = list(df.select_dtypes("datetime64").columns)

# Printing count of each unique value in each categorical column
for column in datetime_col:
    print(df[column].value_counts(normalize = True))
    print("-" * 50)

In [ ]:
# Date Variables to Drop
df = df.drop(['arrival_dt'], axis = 1)
df = df.drop(['death_dttm'], axis = 1)
df = df.drop(['EDArrivalPreviousEDVisit'], axis = 1)


## Treating Duplicates 

In [ ]:
# Check for duplicates
df[df.duplicated()]

In [ ]:
df.duplicated().sum()

In [ ]:
# drop duplicates

df = df.drop_duplicates()

## Imputations for missing data

In [ ]:
# Check Missing Values
df.isna().sum()

In [ ]:
# % Missing Values

(df.isnull().sum() / df.shape[0])*100

In [ ]:
#Missing Categoricals

missingcategoricals = df.columns[df.dtypes == "object"].isna().sum()
missingcategoricals

In [ ]:
# Missing Numerics
missingnumerics =df.select_dtypes(include=np.number).isna().sum()
missingnumerics


## **Collapsing Categoricals and Assigning Dummies**

In [ ]:
# Collapsing Columns, The number is the percentage to group.
cat_col= df.columns[df.dtypes == "object"]
for feature in cat_col:
  df[feature] = df[feature].mask(df[feature].map(df[feature].value_counts(normalize=True)) < 0.0001, 'Other')

In [ ]:
# Dummies for Strings
one_hot = df['Chronic_Condition_Registry_Names'].str.get_dummies(sep=',').add_prefix('CD_')
# Drop column as it is now encoded
df = df.drop('Chronic_Condition_Registry_Names',axis = 1)
# Join the encoded df
df = df.join(one_hot)
df.head()

In [ ]:
# Creating Dummies for columns with strings
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
pd.DataFrame(mlb.fit_transform(df.Active_Mental_Health_Problem_List_Icd_Name.str.split(',')),columns=mlb.classes_,index=df.index).reset_index()

In [ ]:
# Creating dummy variables for the categorical variables
df = pd.get_dummies(df, drop_first = True)

In [ ]:
#To insert the median value of each column into its missing rows:

df.fillna(df.median().round(1), inplace=True)

In [ ]:
# Check Missing Values
df.isna().sum()

In [ ]:
# Drop Anything Missing After Imputations
df = df.drop(['time_to_admit'], axis = 1)

# **Exploratory Data Analysis**

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
# Statistical Summary of Numerical Variables in the data

df.describe().T


### **Univariate analysis**

In [ ]:
# Making a list of all categorical variables
cat_col = list(df.select_dtypes("object").columns)

# Printing count of each unique value in each categorical column
for column in cat_col:
    print(df[column].value_counts(normalize = True))
    print("-" * 50)

In [ ]:
# Checking the number of unique values
df["ID"].nunique()

In [ ]:
# Dropping ID column
data.drop(["ID"], axis = 1, inplace = True)

In [ ]:
# Checking the Outcome of Interest

plt.figure(figsize = (10, 6))

ax = sns.countplot(x = 'Death7Day', data = df)

# Annotating the exact count on the top of the bar for each category 
for p in ax.patches:
    ax.annotate('{:.1f}'.format(p.get_height()), (p.get_x(), p.get_height()+0.35))

In [ ]:
# Make a list of all Numeric Variables

num_cols = df.select_dtypes(include=np.number).columns.tolist()

num_cols

In [ ]:
# Histogram and Bar Graphs for Continous Variables. 

for col in ['age', 'website_visits', 'time_spent_on_website', 'page_views_per_visit']:
    print(col)
    
    print('Skew :',round(data[col].skew(), 2))
    
    plt.figure(figsize = (15, 4))
    
    plt.subplot(1, 2, 1)
    
    data[col].hist(bins = 10, grid = False)
    
    plt.ylabel('count')
    
    plt.subplot(1, 2, 2)
    
    sns.boxplot(x = data[col])
    
    plt.show()

### **Bivariate Analysis**

In [ ]:
#Making a list of all categorical variables 

cat_cols = df.columns[df.dtypes == "object"]

cat_cols


In [ ]:
# Listing Categories Of Categorical variables 
cat_col= df.columns[df.dtypes == "object"]
num_col =df.select_dtypes(include=np.number)

#Printing  count of each unique value in each column
for column in cat_col:
    print (column)
    print(df[column].nunique())
    print(df[column].value_counts())
    print('-'*50)

In [ ]:
# Plot Specific Categoricals by outcome of interest

plt.figure(figsize = (10, 6))

sns.countplot(x = 'current_occupation', hue = 'status', data = data)

plt.show()

In [ ]:
data.groupby(["current_occupation"])["age"].describe()

In [ ]:
# Boxplot of Categorical Variable on X and Continuous on Y

plt.figure(figsize = (10, 5))

sns.boxplot(data["current_occupation"], data["age"])

plt.show()

**Pairwise correlations between all the numerical variables.**

In [ ]:
plt.figure(figsize = (12, 7))

sns.heatmap(df.corr(), annot = True, fmt = ".2f")

plt.show()

In [ ]:
# Histogram of Numeric Variables
#num_col =df.select_dtypes(include=np.number)

df.hist(figsize = (14, 14))

plt.show()

In [ ]:
# Univariate Analysis for Categorical Variables

for i in df.columns[1:]:
    print(df[i].value_counts(normalize = True))
    
    print('*' * 40)

In [ ]:
# Analyze How Outcome Variable is Affected by the Categorical Variables

for i in df.columns[1:]:
    if i != 'ED2Death30Days':
        (pd.crosstab(df[i], df['ED2Death30Days'], normalize = 'index')*100).plot(kind = 'bar', figsize = (8, 4), stacked = True)
        plt.ylabel('ED2Death30Days %')

In [ ]:
# Check the Relationship Between the outcome variable and the numerical variables. 

df.groupby(['ED2Death30Days'])[num_cols].mean()

In [ ]:
# Plotting the correlation between numerical variables
plt.figure(figsize = (15, 8))

sns.heatmap(df[num_cols].corr(), annot = True, fmt = '0.0f', cmap = 'YlGnBu');

In [ ]:
# Univariate Analysis for Categorical Variables.

# Printing the % sub categories of each category.
for i in cat_cols:
    
    print(df[i].value_counts(normalize = True))
    
    print('*' * 40)

In [ ]:
df.columns[1:]

In [ ]:
# Analyze How Outcome Variable is Affected by the Categorical Variables

for i in cat_cols:
    if i != 'Attrition':
        (pd.crosstab(df[i], df['Attrition'], normalize = 'index')*100).plot(kind = 'bar', figsize = (8, 4), stacked = True)
        plt.ylabel('Percentage Attrition %')

In [ ]:
for col in df.columns[1:]:
    print(col)
    
    print('Skew :', round(df[col].skew(), 2))
    
    plt.figure(figsize = (15, 4))
    
    plt.subplot(1, 2, 1)
    
    
    df[col].hist(bins = 20, grid = False)
    
    plt.ylabel('count')
    
    plt.subplot(1, 2, 2)
    
    sns.boxplot(x = df[col])
    
    plt.show()
    

In [ ]:
plt.figure(figsize = (12, 7))

sns.heatmap(df.corr(), annot = False, fmt = ".0f")

plt.show()

# **Principal Component Analysis**

### **Scaling the data**


In [ ]:
scaler = StandardScaler()
data_scaled = pd.DataFrame(scaler.fit_transform(df), columns = df.columns)

In [ ]:
# Defining the number of principal components to generate
n = data_scaled.shape[1]

# Finding principal components for the data
pca = PCA(n_components = n, random_state = 1)

data_pca1 = pd.DataFrame(pca.fit_transform(data_scaled))

# The percentage of variance explained by each principal component
exp_var = pca.explained_variance_ratio_

In [ ]:
df.shape

In [ ]:
# Plotting the explained cumulative variance by principal components
plt.figure(figsize = (10, 10))

plt.plot(range(1, 3112), pca.explained_variance_ratio_.cumsum(), marker = 'o', linestyle = '--')

plt.title("Explained Variances by Components")

plt.xlabel("Number of Components")

plt.ylabel("Cumulative Explained Variance")

plt.show()

In [ ]:
# Finding the least number of components that can explain more than 90% variance
sum = 0

for ix, i in enumerate(exp_var):
    sum = sum + i
    if(sum>0.90):
        print("Number of PCs that explain at least 90% variance: ", ix + 1)
        break

In [ ]:
pc_comps = ['PC1', 'PC2', 'PC3', 'PC4', 'PC5' ]

data_pca = pd.DataFrame(np.round(pca.components_[:5,:], 2), index = pc_comps, columns = data_scaled.columns)

data_pca.T

In [ ]:
def color_high(val):
    if val <= -0.20:
        return 'background: pink'
    
    elif val >= 0.20:
        return 'background: skyblue'   
    
data_pca.T.style.applymap(color_high)

In [ ]:
df_concat = pd.concat([data_pca1, df], axis = 1)
df_concat.head()

In [ ]:
plt.figure(figsize = (7, 7))

sns.scatterplot(data = data_pca1, x = data_pca1[0], y = data_pca1[1])

plt.xlabel("PC1")

plt.ylabel("PC2")

plt.show()

In [ ]:
plt.figure(figsize = (7, 7))

sns.scatterplot(data=df_concat, x = df_concat[0], y = df_concat[1], hue=df_concat['outcome_Yes'])


  # Create a scatter plot with x = 0 and y = 1 using df_concat dataframe

plt.xlabel("PC1")

plt.ylabel("PC2")

plt.show()

**bold text**## t-SNE - T-distributed Stochastic Neighbor Embedding.

What it does: t-SNE is a tool to visualize high-dimensional data. It converts similarities between data points to joint probabilities and tries to minimize the Kullback-Leibler divergence between the joint probabilities of the low-dimensional embedding and the high-dimensional data. t-SNE has a cost function that is not convex, i.e. with different initializations we can get different results.

It is highly recommended to use another dimensionality reduction method (e.g. PCA for dense data or TruncatedSVD for sparse data) to reduce the number of dimensions to a reasonable amount (e.g. 50) if the number of features is very high. This will suppress some noise and speed up the computation of pairwise distances between samples.

### 2d

In [ ]:
tsne = TSNE(n_components = 2, random_state = 1)           # Apply the t-SNE algorithm with random_state = 1

data_tsne = tsne.fit_transform(data_scaled)   # Fit and transform t-SNE function on the scaled data

In [ ]:
data_tsne.shape

In [ ]:
data_tsne = pd.DataFrame(data = data_tsne, columns = ['Component 1', 'Component 2'])

In [ ]:
data_tsne.head()

In [ ]:
sns.scatterplot(x = data_tsne.iloc[:,0], y = data_tsne.iloc[:,1])

plt.show()

In [ ]:
# Let's see the scatter plot by outcome variable
sns.scatterplot(x = data_tsne.iloc[:,0], y = data_tsne.iloc[:,1], hue = df.outcome_Yes)

plt.show()

### 3D

In [ ]:
# Fitting t-SNE with number of components equal to 3 
tsne = TSNE(n_components = 3, random_state = 1)

data_tsne = tsne.fit_transform(data_scaled)

In [ ]:
data_tsne = pd.DataFrame(data_tsne, columns = ['X1', 'X2', 'X3'])

In [ ]:
# Scatter plot for all three components
fig = plt.figure(figsize = (10, 10))

ax = fig.add_subplot(111, projection = '3d')

x = data_tsne['X1']

y = data_tsne['X2']

z = data_tsne['X3']

ax.scatter(x, y, z)

plt.show()

### Different perplexities

In [ ]:
# Visualizing the Data for Different Perplexity Values

In [ ]:
for i in range(10, 50, 5):
    tsne = TSNE(n_components = 2, random_state = 1, perplexity = i)
    
    data_tsne = tsne.fit_transform(data_scaled)
    
    data_tsne = pd.DataFrame(data_tsne)
    
    data_tsne.columns = ['X1', 'X2']
    
    plt.figure(figsize = (7,7))
    
    sns.scatterplot(x = 'X1', y = 'X2', data = data_tsne, hue=df.outcome_Yes)
    
    plt.title("perplexity = {}".format(i))

# BINARY CLASSIFICATION

# **Data preparation for modeling**

## **Separating the independent variables (X) and the dependent variable (Y)**

In [ ]:
# Separating the target variable and other variables
X = df.drop(columns = 'outcome_Yes')

Y = df['outcome_Yes']

In [ ]:
# Scaling the data
sc = StandardScaler()

X_scaled = sc.fit_transform(X)

X_scaled = pd.DataFrame(X_scaled, columns = X.columns)

### **Splitting the data into 70% train and 30% test sets**

Some classification problems can exhibit a large imbalance in the distribution of the target classes: for instance, there could be several times more negative samples than positive samples. In such cases, it is recommended to use the stratified sampling technique to ensure that relative class frequencies are approximately preserved in each train and validation fold.

In [ ]:
# Splitting the data
x_train, x_test, y_train, y_test = train_test_split(X_scaled, Y, test_size = 0.3, random_state = 1, stratify = Y)

**Checking the shape of the train and test data**

In [ ]:
print("Shape of the training set: ", x_train.shape)   

print("Shape of the test set: ", x_test.shape)

print("Percentage of classes in the training set:")

print(y_train.value_counts(normalize = True))

print("Percentage of classes in the test set:")

print(y_test.value_counts(normalize = True))

**Function to calculate and print the classification report and confusion matrix so that we don't have to rewrite the same code repeatedly for each model.**

In [ ]:
# Function to print the classification report and get confusion matrix in a proper format

def metrics_score(actual, predicted):
    print(classification_report(actual, predicted))

    print('ROC AUC is',roc_auc_score(actual, predicted))
    
    cm = confusion_matrix(actual, predicted)

    
    plt.figure(figsize = (8, 5))
    
    sns.heatmap(cm, annot = True,  fmt = '.2f', xticklabels = ['No Outcome', 'Outcome'], yticklabels = ['No Outcome', 'Outcome'])
    
    plt.ylabel('Actual')
    
    plt.xlabel('Predicted')
    
    plt.show()

# LDA


Prepare the data for modeling.
Partition the data into train and test sets.
Build the model on the train data.
Tune the model if required.
Test the data on the test set.

### **Linear Discriminant Analysis**


In [ ]:
# Fitting the LDA model
lda = LinearDiscriminantAnalysis()  # step 1

lda.fit(x_train, y_train)           # step 2 (fitting the object)

In [ ]:
# Checking model performance of LDA
y_pred_train_lda = lda.predict(x_train)  # step 3 (using the fitted object to obtain predictions)

metrics_score(y_train, y_pred_train_lda)

In [ ]:
 #Creating the list of column names
cols = X.columns

# Saving coefficients of LDA model
coef_lda = lda.coef_

# Printing the cofficients of LDA
pd.DataFrame(coef_lda, columns = cols).T.sort_values(by = 0, ascending = False)

#### **Precision-Recall Curve for LDA**

**The Precision-Recall curve summarizes the trade-off between the precision and the recall for a predictive model using different probability thresholds.**

In [ ]:
y_scores_lda = lda.predict_proba(x_train) # predict_proba gives the probability of each observation belonging to each class

precisions_lda, recalls_lda, thresholds_lda = precision_recall_curve(y_train, y_scores_lda[:, 1])

# Plot values of precisions, recalls, and thresholds
plt.figure(figsize = (10, 7))

plt.plot(thresholds_lda, precisions_lda[:-1], 'b--', label = 'precision')

plt.plot(thresholds_lda, recalls_lda[:-1], 'g--', label = 'recall')

plt.xlabel('Threshold')

plt.legend(loc = 'upper left')

plt.ylim([0, 1])

plt.show()

**Let's check the model performance at this threshold**

In [ ]:
optimal_threshold1 = 0.1

y_pred_train_lda = lda.predict_proba(x_train)

metrics_score(y_train, y_pred_train_lda[:,1] > optimal_threshold1)

In [ ]:
# Checking performance on the test data
optimal_threshold1 = 0.1

y_pred_test_lda = lda.predict_proba(x_test)

metrics_score(y_test, y_pred_test_lda[:,1] > optimal_threshold1)

# QDA

### **Quadratic Discriminant Analysis**

Quadratic discriminant analysis (QDA) is a probabilistic parametric classification technique that represents an evolution of LDA for nonlinear class separations. QDA, like LDA, is based on the hypothesis that the probability density distributions are multivariate normal but, in this case, the dispersion is not the same for all of the categories.

In [ ]:
# Fitting QDA model
qda = QuadraticDiscriminantAnalysis()

qda.fit(x_train, y_train)

In [ ]:
# Checking model performance on the training data
y_pred_train_qda = qda.predict(x_train)

metrics_score(y_train, y_pred_train_qda)

In [ ]:
sklearn.metrics.roc_auc_score()

In [ ]:
# Checking performance of the model on the test data
y_pred_test_qda = qda.predict(x_test)

metrics_score(y_test, y_pred_test_qda)

# Logistic Model

### **Logistic Regression**

Logistic Regression is a supervised learning algorithm, generally used for binary classification problems, i.e., where the dependent variable is categorical and has only two possible values. In logistic regression, we use the sigmoid function to calculate the probability of an event Y, given some features X as:

                                    P(Y)=1/(1 + exp(-X))

In [ ]:
# Fitting the logistic regression model
lg = LogisticRegression()

lg.fit(x_train,y_train)

In [ ]:
# Checking the performance on the training data
y_pred_train = lg.predict(x_train)

metrics_score(y_train, y_pred_train)

In [ ]:
# Checking the performance on the test dataset
y_pred_test = lg.predict(x_test)
metrics_score(y_test, y_pred_test)

In [ ]:
# Printing the coefficients of logistic regression
cols = X.columns

coef_lg = lg.coef_

pd.DataFrame(coef_lg,columns = cols).T.sort_values(by = 0, ascending = False)

The coefficients of the logistic regression model give us the log of odds, which is hard to interpret in the real world. We can convert the log of odds into odds by taking its exponential.

In [ ]:
odds = np.exp(lg.coef_[0]) # Finding the odds

# Adding the odds to a DataFrame and sorting the values
pd.DataFrame(odds, x_train.columns, columns = ['odds']).sort_values(by = 'odds', ascending = False) 

**The Precision-Recall Curve for Logistic Regression**

In [ ]:
y_scores_lg = lg.predict_proba(x_train) # predict_proba gives the probability of each observation belonging to each class


precisions_lg, recalls_lg, thresholds_lg = precision_recall_curve(y_train, y_scores_lg[:, 1])

# Plot values of precisions, recalls, and thresholds
plt.figure(figsize = (10, 7))

plt.plot(thresholds_lg, precisions_lg[:-1], 'b--', label = 'precision')

plt.plot(thresholds_lg, recalls_lg[:-1], 'g--', label = 'recall')

plt.xlabel('Threshold')

plt.legend(loc = 'upper left')

plt.ylim([0, 1])

plt.show()

In [ ]:
#optimal threshold on training data

In [ ]:
optimal_threshold1 = .3

y_pred_train = lg.predict_proba(x_train)

metrics_score(y_train, y_pred_train[:, 1] > optimal_threshold1)

In [ ]:
# ROC Curve
#define metrics
fpr, tpr, _ = metrics.roc_curve(y_train, y_pred_train[:, 1] > optimal_threshold1)
auc = metrics.roc_auc_score(y_train, y_pred_train[:, 1] > optimal_threshold1)

#create ROC curve
plt.plot(fpr,tpr,label="AUC="+str(auc))
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc=4)
plt.show()

In [ ]:
# optimal threshold on test data

optimal_threshold1 = .3

y_pred_test = lg.predict_proba(x_test)

metrics_score(y_test, y_pred_test[:, 1] > optimal_threshold1)

In [ ]:
# ROC Curve
#define metrics
fpr, tpr, _ = metrics.roc_curve(y_test, y_pred_test[:, 1] > optimal_threshold1)
auc = metrics.roc_auc_score(y_test, y_pred_test[:, 1] > optimal_threshold1)

#create ROC curve
plt.plot(fpr,tpr,label="AUC="+str(auc))
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc=4)
plt.show()

# **K-Nearest Neighbors (K-NN)**

K-NN uses features from the training data to predict the values of new data points, which means the new data point will be assigned a value based on how similar it is to the data points in the training set.

The following steps are performed in K-NN:

Select K
Calculate distance (Euclidean, Manhattan, etc.)
Find the K closest neighbors
Take majority vote for labels
The “K” in the K-NN algorithm is the number of nearest neighbors we wish to take the vote from. Generally, K is taken to be an odd number when the number of classes is even, so as to get a majority vote. Let's say K=3. In that case, we will make a circle with the new data point as the center just as big as enclosing only the three nearest data points on the plane.

But before actually building the model, we need to identify the value of K to be used in K-NN. We will perform the following steps for the same.

For every value of K (from 1 to 15), split the training set into a new train and validation sets (30 times)
Scale the training data and the validation data
Take the average of the error on these training and the validation sets for each value of K
Plot the average train vs validation error for all Ks
Choose the optimal K from the plot where the two errors are comparable

Find where the test error (error on validation data) does not increase
Find where the traning error does not increase. 
Pick that K


## Grid Search to Find value of K and Other Hyperparameters


Points to note:

Hyperparameter tuning is tricky in the sense that there is no direct way to calculate how a change in the hyperparameter value will reduce the loss of your model, so we usually resort to experimentation.

Grid search is a tuning technique that attempts to compute the optimum values of hyperparameters.

Grid search is an exhaustive search of values that tries many iterations to compute the optimum values of hyperparameters. So, it might take up to 30 minutes for the code to run depending on the number of values and hyperparameters passed.

The hyperparameters that we are tuning are:

n_neighbors: Number of neighbors to use.

weights={'uniform', 'distance'}

uniform : uniform weights. All points in each neighborhood are weighted equally.
distance : weight points by the inverse of their distance. In this case, closer neighbors of a query point will have a greater influence than neighbors that are further away.
p: When p = 1, this is equivalent to using Manhattan_distance (L1), and Euclidean_distance (L2) is used for p = 2.

In [ ]:
# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.30, random_state = 1, stratify = Y)

In [ ]:
# Scaling the data
scaler = StandardScaler()

# fit_transform the training data
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)

# Transform the testing data
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns) 

## GRID Search Takes a Long Time

In [ ]:
# Grid Search. This takes a long time. 
knn = KNeighborsClassifier()

params_knn = {'n_neighbors':np.arange(2, 20, 2), 'weights':['uniform','distance'], 'p':[1, 2]}

grid_knn = GridSearchCV(estimator = knn, param_grid = params_knn, scoring = 'f1', cv = 10)

model_knn = grid_knn.fit(X_train_scaled,y_train)

knn_estimator = model_knn.best_estimator_

print(knn_estimator)

In [ ]:
# Define K-NN model

knn = KNeighborsClassifier(n_neighbors=5)

In [ ]:
# Fitting data to the K-NN model

knn.fit(x_train,y_train)

In [ ]:
# Checking the performance of K-NN model on the training data
y_pred_train_knn = knn.predict(x_train)

metrics_score(y_train, y_pred_train_knn)

In [ ]:
# Checking the performance of K-NN model on the testing data
y_pred_test_knn = knn.predict(x_test)

metrics_score(y_test, y_pred_test_knn)

# **Decision Tree Model**

### **Building a Decision Tree Model**

* We will build our model using the **DecisionTreeClassifier function.** 
* If the frequency of class A is 17% and the frequency of class B is 83%, then class B will become the dominant class and **the decision tree will become biased toward the dominant class.**
* **class_weight is a hyperparameter** for the decision tree classifier, and in this case, we can pass a dictionary {0:0.17, 1:0.83} to the model to specify the weight of each class and the decision tree will give more weightage to class 1.

We will use the class_weight hyperparameter with the value equal to {0: 0.3, 1: 0.7} which is approximately the opposite of the imbalance in the original data. 

**This would tell the model that 1 is the important class here.**

In [ ]:
# Describe Feature of Interest
df.outcome_Yes.describe()

In [ ]:
# Building decision tree model
dt = DecisionTreeClassifier(class_weight = {0: 0.11, 1: 0.89}, random_state = 1)

In [ ]:
# Fitting decision tree model
dt.fit(x_train, y_train)

In [ ]:
# Checking performance on the training dataset
y_train_pred_dt = dt.predict(x_train)

metrics_score(y_train, y_train_pred_dt)

In [ ]:
# ROC Curve
#define metrics
fpr, tpr, _ = metrics.roc_curve(y_train, y_train_pred_dt)
auc = metrics.roc_auc_score(y_train, y_train_pred_dt)

#create ROC curve
plt.plot(fpr,tpr,label="AUC="+str(auc))
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc=4)
plt.show()

In [ ]:
# Checking performance on the test dataset
y_test_pred_dt = dt.predict(x_test)

metrics_score(y_test, y_test_pred_dt)

In [ ]:
# ROC Curve
#define metrics
fpr, tpr, _ = metrics.roc_curve(y_test, y_test_pred_dt)
auc = metrics.roc_auc_score(y_test, y_test_pred_dt)

#create ROC curve
plt.plot(fpr,tpr,label="AUC="+str(auc))
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc=4)
plt.show()

In [ ]:
# Plot the feature importance

importances = dt.feature_importances_

columns = X.columns

importance_df = pd.DataFrame(importances, index = columns, columns = ['Importance']).sort_values(by = 'Importance', ascending = False)

plt.figure(figsize = (13, 13))

sns.barplot(importance_df.Importance,importance_df.index)

In [ ]:
importance_df

In [ ]:
# Plot the Tree
features = list(X.columns)

plt.figure(figsize = (30, 20))

tree.plot_tree(dt, max_depth = 4, feature_names = features, filled = True, fontsize = 12, node_ids = True, class_names = True)

plt.show()

In [ ]:
# Decision Tree Depth vs Error on test and train data

train_scores = []

test_scores = []

for depth in range(1, 10):
    
    clf = tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = depth)
    
    clf = clf.fit(x_train, y_train)
    
    train_scores.append(1 - clf.score(x_train, y_train))
    
    test_scores.append(1 - clf.score(x_test, y_test))

plt.plot(range(1, 10), train_scores, '-o', label = "train")

plt.plot(range(1, 10), test_scores, '-o', label = "test")

plt.legend(loc = 'best')

plt.xlabel('max depth')

plt.ylabel('misclassification error (pcnt)')

# Setting the range of the Y-axis
plt.ylim(0, 0.3)

plt.title(f"Decision Tree")

fig = plt.gcf()

fig.set_size_inches(10, 6)

plt.show()

### **Tuning Models**

Using GridSearch for Hyperparameter Tuning
Hyperparameter tuning is tricky in the sense that there is no direct way to calculate how a change in the hyperparameter value will reduce the loss of your model, so we usually resort to experimentation. We'll use Grid search to perform hyperparameter tuning.
Grid search is a tuning technique that attempts to compute the optimum values of hyperparameters.
It is an exhaustive search that is performed on the specific parameter values of a model.
The parameters of the estimator/model used to apply these methods are optimized by cross-validated grid-search over a parameter grid.

Criterion{“gini”, “entropy”}

The function is to measure the quality of a split. Supported criteria are “gini” for the Gini impurity and “entropy” for the information gain.

max_depth

The maximum depth of the tree. If None, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples.

min_samples_leaf: The minimum number of samples is required to be at a leaf node. A split point at any depth will only be considered if it leaves at least min_samples_leaf training samples in each of the left and right branches. This may have the effect of smoothing the model, especially in regression.

### **Decision Tree - Hyperparameter Tuning**

We will use the class_weight hyperparameter with the value equal to {0: 0.3, 1: 0.7} which is approximately the opposite of the imbalance in the original data. 

**This would tell the model that 1 is the important class here.**

In [ ]:
# Choose the type of classifier
dtree_estimator = DecisionTreeClassifier(class_weight = {0: 0.11, 1: 0.89}, random_state = 1)

# Grid of parameters to choose from
parameters = {'max_depth': np.arange(2, 7), 
              'criterion': ['gini', 'entropy'],
              'min_samples_leaf': [5, 10, 20, 25]
             }

# Type of scoring used to compare parameter combinations
scorer = metrics.make_scorer(recall_score, pos_label = 1)

# Run the grid search
gridCV = GridSearchCV(dtree_estimator, parameters, scoring = scorer, cv = 10)

# Fitting the grid search on the train data
gridCV = gridCV.fit(x_train, y_train)

# Set the classifier to the best combination of parameters
dtree_estimator = gridCV.best_estimator_

# Fit the best estimator to the data
dtree_estimator.fit(x_train, y_train)

In [ ]:
# Checking performance on the training dataset
y_train_pred_dt = dtree_estimator.predict(x_train)

metrics_score(y_train, y_train_pred_dt)

In [ ]:
# Checking performance on the test dataset
y_test_pred_dt = dtree_estimator.predict(x_test)

metrics_score(y_test, y_test_pred_dt)

In [ ]:
importances = dtree_estimator.feature_importances_

columns = X.columns

importance_df = pd.DataFrame(importances, index = columns, columns = ['Importance']).sort_values(by = 'Importance', ascending = False)

plt.figure(figsize = (13, 13))

sns.barplot(importance_df.Importance, importance_df.index)

In [ ]:
importance_df

In [ ]:
# Plot the Tree

features = list(X.columns)

plt.figure(figsize = (30, 20))

tree.plot_tree(dt, max_depth = 4, feature_names = features, filled = True, fontsize = 12, node_ids = True, class_names = True)

plt.show()



# Boosting

good article - https://medium.com/@dmahidhara/dealing-with-rare-events-in-machine-learning-14c2c167e222 

## ADA Boost

If the AdaBoost ensemble is overfitting the training set, you can try reducing the number of estimators or more strongly regularizing the base estimator. 

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1),n_estimators=200,
    algorithm="SAMME.R", learning_rate=1, random_state = 0)
ada_clf.fit(x_train, y_train)


In [ ]:
# Checking performance on the training dataset
y_train_pred_ada_clf = ada_clf.predict(x_train)

metrics_score(y_train, y_train_pred_ada_clf)

In [ ]:
# Checking performance on the test dataset
y_test_pred_ada_clf = ada_clf.predict(x_test)

metrics_score(y_test, y_test_pred_ada_clf)

## Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
# Build Gradient Boost Model
gbclf = GradientBoostingClassifier(n_estimators = 100, learning_rate=1.0, max_depth=1,gbrandom_state=0)

In [ ]:
# Fit the model
gbclf.fit(x_train, y_train)

In [ ]:
# Checking performance on the training dataset
y_train_pred_gbclf = gbclf.predict(x_train)

metrics_score(y_train, y_train_pred_gbclf)

In [ ]:
# Checking performance on the test dataset
y_test_pred_gbclf = gbclf.predict(x_test)

metrics_score(y_test, y_test_pred_gbclf)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
# Fitting the Gradient Boost classifier on the training data
gb_estimator = GradientBoostingClassifier(learning_rate=0.1, min_samples_split=5,min_samples_leaf=5,max_depth=8,max_features='sqrt',subsample=0.8,random_state=1)

In [ ]:
#Grid Search Gradient Boost

parameters = {
    "loss":["deviance"],
    "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    "min_samples_split": np.linspace(0.1, 0.5, 12),
    "min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_depth":[3,5,8],
    "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse",  "mae"],
    "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators":[10]
    }

clf = GridSearchCV(GradientBoostingClassifier(), parameters, cv=10, n_jobs=-1)

clf.fit(x_train, y_train)
print(clf.score(x_train, y_train))
print(clf.best_params_)


In [ ]:
gb_estimator.fit(x_train, y_train)

In [ ]:
# Checking performance on the training data
y_pred_train_gb = gb_estimator.predict(x_train)

metrics_score(y_train, y_pred_train_gb)

In [ ]:
# ROC Curve
#define metrics
fpr, tpr, _ = metrics.roc_curve(y_train,  y_pred_train_gb)
auc = metrics.roc_auc_score(y_train, y_pred_train_gb)

#create ROC curve
plt.plot(fpr,tpr,label="AUC="+str(auc))
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc=4)
plt.show()

In [ ]:
# Checking performance on the testing data
y_pred_test_gb = gb_estimator.predict(x_test)

metrics_score(y_test, y_pred_test_gb)

In [ ]:
# ROC Curve
#define metrics
fpr, tpr, _ = metrics.roc_curve(y_test, y_pred_test_gb)
auc = metrics.roc_auc_score(y_test, y_pred_test_gb)

#create ROC curve
plt.plot(fpr,tpr,label="AUC="+str(auc))
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc=4)
plt.show()

## XGBOOST

In [ ]:
## LCE - Local Cascade Ensemble

https://towardsdatascience.com/random-forest-or-xgboost-it-is-time-to-explore-lce-2fed913eafb8

In [ ]:
pip install lcensemble

In [ ]:
conda install -c conda-forge lcensemble

In [ ]:
from lce import LCEClassifier
from sklearn.datasets import load_iris
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score, train_test_split

In [ ]:
# Load data and generate a train/test split
data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(data.data, 
                                                    data.target, 
                                                    random_state=0)

# Train LCEClassifier with default parameters
clf = LCEClassifier(n_jobs=-1, random_state=123)
clf.fit(X_train, y_train)

# Make prediction and generate classification report
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
# Perform cross-validation with scikit-learn model selection tool 
print(cross_val_score(clf, data.data, data.target, cv=3))

In [ ]:
import numpy as np
from lce import LCEClassifier
from sklearn.datasets import load_iris
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [ ]:
# Load data and generate a train/test split
data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(data.data, 
                                                    data.target, 
                                                    random_state=0)

# Input 20% of missing values per variable in the train set
np.random.seed(0)
m = 0.2
for j in range(0, X_train.shape[1]):
    sub = np.random.choice(X_train.shape[0], int(X_train.shape[0]*m))
    X_train[sub, j] = np.nan

# Train LCEClassifier with default parameters
clf = LCEClassifier(n_jobs=-1, random_state=123)
clf.fit(X_train, y_train)

# Make prediction and generate classification report
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

#  Random Forest Classifier

**Random Forest is a bagging algorithm where the base models are Decision Trees.** Samples are taken from the training data and on each sample, a decision tree makes a prediction. 

**The results from all the decision trees are combined and the final prediction is made using voting (for classification problems) or averaging (for regression problems).**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

train_scores = []

test_scores = []

for depth in range(1, 10):
    
    clf = RandomForestClassifier(random_state = 0, criterion = 'entropy', max_depth = depth, n_estimators = 25)
    
    clf = clf.fit(x_train, y_train)
    
    train_scores.append(1 - clf.score(x_train, y_train))
    
    test_scores.append(1 - clf.score(x_test, y_test))

plt.plot(range(1, 10), train_scores, '-o', label = "train")

plt.plot(range(1, 10), test_scores, '-o', label = "test")

plt.legend(loc = 'upper right')

plt.xlabel('max depth')

plt.ylim(0, 0.3)

plt.ylabel('misclassification error (pcnt)')

plt.title(f"Random Forest with 25 estimators")

fig = plt.gcf()

fig.set_size_inches(10, 6)

plt.show()

In [ ]:
# Fitting the Random Forest classifier on the training data
rf_estimator = RandomForestClassifier(class_weight = {0: 0.12, 1: 0.88}, random_state = 1)

rf_estimator.fit(x_train, y_train)

In [ ]:
# Checking performance on the training data
y_pred_train_rf = rf_estimator.predict(x_train)

metrics_score(y_train, y_pred_train_rf)

In [ ]:
# Checking performance on the testing data
y_pred_test_rf = rf_estimator.predict(x_test)

metrics_score(y_test, y_pred_test_rf)

**Let's check the feature importance of the Random Forest**

In [ ]:
importances = rf_estimator.feature_importances_

columns = X.columns

importance_df = pd.DataFrame(importances, index = columns, columns = ['Importance']).sort_values(by = 'Importance', ascending = False)

plt.figure(figsize = (21, 21))

sns.barplot(importance_df.Importance, importance_df.index)

In [ ]:
importance_df

### **Tuning Hyperparameters of the Random Forest classifier**

**n_estimators**: The number of trees in the forest.

**min_samples_split**: The minimum number of samples required to split an internal node.

**min_samples_leaf**: The minimum number of samples required to be at a leaf node. 

**max_features{“auto”, “sqrt”, “log2”, 'None'}**: The number of features to consider when looking for the best split.

- If “auto”, then max_features=sqrt(n_features).

- If “sqrt”, then max_features=sqrt(n_features) (same as “auto”).

- If “log2”, then max_features=log2(n_features).

- If None, then max_features=n_features.

In [ ]:
# Choose the type of classifier
rf_estimator_tuned = RandomForestClassifier(class_weight = {0: 0.12, 1: 0.88}, random_state = 1)

# Grid of parameters to choose from
params_rf = {  
        "n_estimators": [100, 250, 500],
        "min_samples_leaf": np.arange(1, 4, 1),
        "max_features": [0.7, 0.9, 'auto'],
}


# Type of scoring used to compare parameter combinations - recall score for class 1
scorer = metrics.make_scorer(recall_score, pos_label = 1)

# Run the grid search
grid_obj = GridSearchCV(rf_estimator_tuned, params_rf, scoring = scorer, cv = 5)

grid_obj = grid_obj.fit(x_train, y_train)

# Set the classifier to the best combination of parameters
rf_estimator_tuned = grid_obj.best_estimator_

In [ ]:
# Fitting the best algorithm to the training data
rf_estimator_tuned.fit(x_train, y_train)

In [ ]:
# Checking performance on the training data
y_pred_train_rf_tuned = rf_estimator_tuned.predict(x_train)

metrics_score(y_train, y_pred_train_rf_tuned)

In [ ]:
# Checking performance on the test data
y_pred_test_rf_tuned = rf_estimator_tuned.predict(x_test)

metrics_score(y_test, y_pred_test_rf_tuned)

In [ ]:
# Plotting feature importance
importances = rf_estimator_tuned.feature_importances_

columns = X.columns

importance_df = pd.DataFrame(importances, index = columns, columns = ['Importance']).sort_values(by = 'Importance', ascending = False)

plt.figure(figsize = (13, 13))

sns.barplot(importance_df.Importance, importance_df.index)

### **Tune Random Forest Using GridSearch**
- **Tune the random forest classifier using GridSearchCV (2 Marks)**
- **Check the performance on both training and testing datasets (2 Marks)**
- **Compare the results with the results from the random forest model with default parameters and write your observations (2 Marks)**

In [ ]:
# Choose the type of classifier 
rf_estimator_tuned = RandomForestClassifier(criterion = "entropy", random_state = 1)

# Grid of parameters to choose from
parameters = {"n_estimators": [110, 120],
    "max_depth": [6, 7],
    "min_samples_leaf": [20, 25],
    "max_features": [0.8, 0.9],
    "max_samples": [0.9, 1],
    "class_weight": [{0: 0.7, 1: 0.3}, "balanced", {0: 0.4, 1: 0.1}]
             }

# Type of scoring used to compare parameter combinations - recall score for class 1
scorer = metrics.make_scorer(recall_score, pos_label = 1)

# Run the grid search on the training data using scorer=scorer and cv=5
grid_obj = GridSearchCV(rf_estimator_tuned, parameters, scoring = scorer, cv = 5)

grid_obj = grid_obj.fit(X_train, y_train)

# Save the best estimator to variable rf_estimator_tuned
rf_estimator_tuned = grid_obj.best_estimator_

#Fit the best estimator to the training data
rf_estimator_tuned.fit(X_train, y_train)

# Neural Networks

In [ ]:
## **Loading the libraries and the dataset**

We will be using **Google Colab** to run this notebook. 

First, let's **import the data** so that Colab can access the dataset. One way to load the data in Colab is by uploading the dataset directly in the notebook. The following code does just that.
Once you run the cell, it would ask you to choose the file from your local system.

In [ ]:
import numpy as np

import pandas as pd

import matplotlib.pyplot as plt

import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf

from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense, Dropout, Activation

In [ ]:
df.head()

In [ ]:
# Converting the target variable into a categorical variable
#df['Admit'] = data['Chance of Admit '].apply(lambda x: 1 if x > 0.8 else 0)

In [ ]:
# Dropping columns
#data = data.drop(['Serial No.', 'Chance of Admit '], axis = 1)

#data.head()

In [ ]:
df.describe().T

## Split, Scale, Normalize

Now, let's split the dataset into train and test datasets. To do that, we would be extracting all the **independent variables** and saving them into a variable **features**. And the target variable **Admit** would be saved into a variable **target**. These two variables will be used to split the parent dataset into train and test datasets.

In [ ]:
features = df.drop(['outcome_Yes'], axis = 1)

target = df['outcome_Yes']

In [ ]:
# Splitting the dataset into the train and the test data
X_train, X_test, y_train, y_test =  train_test_split(features, target, test_size = 0.3, random_state = 42)

In [ ]:
scaler = MinMaxScaler()

# Here, we are passing all the features (numerical and categorical), that's okay as min-max scaler will not change values of categorical variables
X_train_normalized = scaler.fit_transform(X_train)

In [ ]:
X_test_normalized = scaler.transform(X_test)

## Model Building

In neural networks, there are so many hyper-parameters that you can play around with and tune the network to get the best results. Some of them are: 


1.   Number of hidden layers
2.   Number of neurons in each hidden layer
3.   Activation functions in hidden layers
4.   Optimizers
5.   Random initialization of weights and biases
6.   Batch size
7.   Learning rate
8.   Early stopping
9.   L1 and L2 Regularization
10.  Dropout
11.  Momentum

and so on...

First, let's set the seed for random number generators in NumPy, Python, and TensorFlow to be able to reproduce the same results every time we run the code.

In [ ]:
# Fixing the seed for random number generators
np.random.seed(42)

import random

random.seed(42)

tf.random.set_seed(42)

In [ ]:
#(rows,columns)
features.shape

Let's build a feed-forward neural network with 2 hidden layers and the output layer.

In [ ]:
# We will be adding the layers sequentially
model_1 = Sequential()

# First hidden layer with 128 neurons and relu activation function, the input_shape tuple denotes number of independent variables
model_1.add(Dense(128, activation = 'relu', input_shape = (1774, )))

# We will be switching 20% of neurons randomly at each iteration to avoid overfitting
model_1.add(Dropout(0.2))

# Second hidden layer with 64 neurons and relu activation function
model_1.add(Dense(64, activation = 'relu'))

# We will be switching 10% of neurons off randomly at each iteration to avoid 
model_1.add(Dropout(0.1))

# Output layer with only one neuron and sigmoid as activation function which will give the probability of students getting admitted into UCLA  
model_1.add(Dense(1, activation = 'sigmoid'))

Once we are done with the model architecture, we need to compile the model, where we need to provide the loss function that we want to optimize, the optimization algorithm, and the evaluation metric that we are interested in to evaluate the model.

Since this is a binary classification task, we will be minimizing the **binary_crossentropy** and we can choose one optimizer out of 
1.   SGD
2.   RMSprop
3.   Adam
4.   Adadelta
5.   Adagrad
6.   Adamax
7.   Nadam
8.   Ftrl

This is a hyper-parameter. You can play around with these optimizers to check which one performs better with a particular data.

For now, let's try **adamax** optimizer with **accuracy** as the metric and see the model's summary.

In [ ]:
model_1.compile(loss = 'binary_crossentropy', optimizer = 'adamax', metrics = ['accuracy'])

model_1.summary()

### Training the Model


Let's now train the model using the below piece of code. We will keep 10% of the training data for validation.

In [ ]:
history_1 = model_1.fit(X_train_normalized, 
                        
                    y_train,

                    validation_split = 0.1, 

                    epochs = 20, 

                    verbose = 2)

### **Plotting Accuracy vs Epoch Curve**

In [ ]:
plt.plot(history_1.history['accuracy'])

plt.plot(history_1.history['val_accuracy'])

plt.title('Accuracy vs Epochs')

plt.ylabel('Accuracy')

plt.xlabel('Epoch')

plt.legend(['Train', 'Validation'], loc = 'lower right')

plt.show()

### **HyperParameter Tuning**

let's try to change the number of hidden layers, the number of neurons in each hidden layer, the activation function in the hidden layer, and the optimizer from **adamax** to **adam**. Also, we have observed that validation accuracy became constant after some epochs, let's try less number of epochs which would also reduce the computation time. 

In [ ]:
# Clearing the backend
from tensorflow.keras import backend

backend.clear_session()

In [ ]:
# Fixing the seed for random number generators
np.random.seed(42)

import random

random.seed(42)

tf.random.set_seed(42)

In [ ]:
model_2 = Sequential()

model_2.add(Dense(128, activation = 'relu', input_shape = (3110, )))

model_2.add(Dropout(0.3))

model_2.add(Dense(64, activation = 'relu'))

model_2.add(Dropout(0.3))

model_2.add(Dense(32, activation = 'relu'))

model_2.add(Dense(1, activation = 'sigmoid'))

In [ ]:
model_2.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

model_2.summary()

In [ ]:
history_2 = model_2.fit(X_train_normalized,
                         
                    y_train, 

                    validation_split = 0.1,

                    epochs = 20,

                    verbose = 2)

In [ ]:
plt.plot(history_2.history['accuracy'])

plt.plot(history_2.history['val_accuracy'])

plt.title('Accuracy vs Epochs')

plt.ylabel('Accuracy')

plt.xlabel('Epoch')

plt.legend(['Train', 'Validation'], loc = 'lower right')

plt.show()

In [ ]:
plt.plot(history_1.history['loss'])

plt.plot(history_1.history['val_loss'])

plt.title('Loss vs Epochs')

plt.ylabel('Loss')

plt.xlabel('Epoch')

plt.legend(['Train', 'Validation'], loc = 'upperright')

plt.show()

 ### Model 3 - Change Learning *Rate*

In [ ]:
# Clearing the backend
from tensorflow.keras import backend

backend.clear_session()

In [ ]:
# Fixing the seed for random number generators
np.random.seed(42)

import random

random.seed(42)

tf.random.set_seed(42)

In [ ]:
df.shape

In [ ]:
model_3 = Sequential()

model_3.add(Dense(512, activation = 'relu', input_shape = (8990, )))

model_3.add(Dropout(0.2))

model_3.add(Dense(256, activation = 'relu'))

model_3.add(Dropout(0.2))

model_3.add(Dense(128, activation = 'relu'))

model_3.add(Dropout(0.2))

model_3.add(Dense(64, activation = 'relu'))

model_3.add(Dropout(0.2))

model_3.add(Dense(32, activation = 'relu'))



model_3.add(Dense(1, activation = 'sigmoid'))

Metrics: accuracy, AUC, Precision, Recall, F1Score

In [ ]:
model_3.compile(loss = 'binary_crossentropy', optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0005), metrics = ['accuracy'])

model_3.summary()

In [ ]:
history_3 = model_3.fit(X_train_normalized, 
                        
                    y_train,

                    validation_split = 0.1,

                    epochs = 20,

                    verbose = 2)

In [ ]:
plt.plot(history_3.history['accuracy'])

plt.plot(history_3.history['val_accuracy'])

plt.title('Accuracy vs Epochs')

plt.ylabel('Accuracy')

plt.xlabel('Epoch')

plt.legend(['Train', 'Validation'], loc  = 'lower right')

plt.show()

In [ ]:
plt.plot(history_3.history['loss'])

plt.plot(history_3.history['val_loss'])

plt.title('Loss vs Epochs')

plt.ylabel('Loss')

plt.xlabel('Epoch')

plt.legend(['Train', 'Validation'], loc = 'upperright')

plt.show()

## **Model evaluation on the test data**

In [ ]:
model_3.evaluate(X_test_normalized, y_test, verbose = 1)

test_pred = np.round(model_3.predict(X_test_normalized))

In [ ]:
from sklearn.metrics import classification_report

from sklearn.metrics import roc_auc_score

from sklearn.metrics import confusion_matrix

print(classification_report(y_test, test_pred))

print('ROC AUC is',roc_auc_score(y_test, test_pred))

cm = confusion_matrix(y_test, test_pred)

plt.figure(figsize = (8, 5))

sns.heatmap(cm, annot = True,  fmt = '.0f', xticklabels = ['Outcome', 'No Outcome'], yticklabels = ['Outcome', 'No Outcome'])

plt.ylabel('Actual')

plt.xlabel('Predicted')

plt.show()